In [7]:
# ipl_analysis_excel.py
# Complete IPL Project (Excel Version)
# Compatible with: matches (1).xlsx and deliveries (1).xlsx

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------- FILE PATHS --------------------
MATCHES_XLSX = r"C:\Users\Thota Rajasekhar\Downloads\matches (1).xlsx"
DELIVERIES_XLSX = r"C:\Users\Thota Rajasekhar\Downloads\deliveries (1).xlsx"

# -------------------- SETUP --------------------
PLOTS_DIR = "plots"
os.makedirs(PLOTS_DIR, exist_ok=True)
plt.rcParams["figure.figsize"] = (10, 6)

def save_plot(fig, name):
    path = os.path.join(PLOTS_DIR, name)
    fig.savefig(path, bbox_inches="tight")
    print(f"✅ Plot saved: {path}")

# -------------------- 1. LOAD & CLEAN DATA --------------------
def load_and_clean():
    matches = pd.read_excel(MATCHES_XLSX)
    deliveries = pd.read_excel(DELIVERIES_XLSX)

    matches.columns = matches.columns.str.strip()
    deliveries.columns = deliveries.columns.str.strip()

    if "date" in matches.columns:
        matches["date"] = pd.to_datetime(matches["date"], errors="coerce")

    for df in (matches, deliveries):
        for col in df.select_dtypes(include="object").columns:
            df[col] = df[col].astype(str).str.strip()

    matches.fillna({"city": "Unknown", "winner": "No Result", "venue": "Unknown"}, inplace=True)
    deliveries.fillna("", inplace=True)

    df = deliveries.merge(matches, left_on="match_id", right_on="id", how="left")
    print(f"Matches: {matches.shape}, Deliveries: {deliveries.shape}, Combined: {df.shape}")
    return matches, deliveries, df

# -------------------- 2. HIGHEST & LOWEST SCORERS --------------------
def top_batsmen(df, top_n=15):
    result = df.groupby("batsman")["batsman_runs"].sum().sort_values(ascending=False).head(top_n)
    fig, ax = plt.subplots()
    result.sort_values().plot(kind="barh", ax=ax, color="orange")
    ax.set_title("Top Batsmen by Total Runs")
    ax.set_xlabel("Runs")
    ax.set_ylabel("Batsman")
    save_plot(fig, "top_batsmen.png")
    plt.close(fig)
    return result

# -------------------- 3. TOP BOWLERS --------------------
def top_bowlers(df, top_n=15):
    wickets = df[df["player_dismissed"] != ""].groupby("bowler")["player_dismissed"].count().sort_values(ascending=False)
    top = wickets.head(top_n)
    fig, ax = plt.subplots()
    top.sort_values().plot(kind="barh", ax=ax, color="skyblue")
    ax.set_title("Top Bowlers by Wickets")
    ax.set_xlabel("Wickets")
    ax.set_ylabel("Bowler")
    save_plot(fig, "top_bowlers.png")
    plt.close(fig)
    return top

# -------------------- 4. ZERO WICKET BOWLERS --------------------
def zero_wicket_bowlers(df):
    all_bowlers = df["bowler"].unique()
    wickets = df[df["player_dismissed"] != ""].groupby("bowler")["player_dismissed"].count()
    zero_wicket = [b for b in all_bowlers if b not in wickets.index]
    return pd.Series(zero_wicket, name="Zero Wicket Bowlers")

# -------------------- 5. TOP FIELDERS --------------------
def top_fielders(df, top_n=10):
    fielders = df[df["dismissal_kind"].isin(["caught", "run out", "stumped"])]
    result = fielders.groupby("fielder")["player_dismissed"].count().sort_values(ascending=False).head(top_n)
    fig, ax = plt.subplots()
    result.sort_values().plot(kind="barh", ax=ax, color="green")
    ax.set_title("Top Fielders (Catches + Run-Outs)")
    ax.set_xlabel("Dismissals")
    ax.set_ylabel("Fielder")
    save_plot(fig, "top_fielders.png")
    plt.close(fig)
    return result

# -------------------- 6. TOSS WINNER VS MATCH WINNER --------------------
def toss_vs_match(matches):
    matches["toss_match_result"] = np.where(matches["toss_winner"] == matches["winner"], "Won Toss & Match", "Lost After Toss")
    result = matches["toss_match_result"].value_counts()
    fig, ax = plt.subplots()
    result.plot.pie(autopct="%1.1f%%", startangle=90, colors=["gold", "lightgray"], ax=ax)
    ax.set_ylabel("")
    ax.set_title("Toss Winner vs Match Winner")
    save_plot(fig, "toss_vs_match.png")
    plt.close(fig)
    return result

# -------------------- 7. MOST SUCCESSFUL TEAMS --------------------
def team_wins(matches):
    wins = matches["winner"].value_counts()
    fig, ax = plt.subplots()
    wins.sort_values().plot(kind="barh", ax=ax, color="purple")
    ax.set_title("Most Successful IPL Teams")
    ax.set_xlabel("Wins")
    ax.set_ylabel("Team")
    save_plot(fig, "team_wins.png")
    plt.close(fig)
    return wins

# -------------------- 8. SIXES & FOURS --------------------
def fours_sixes(df, top_n=10):
    df["four"] = np.where(df["batsman_runs"] == 4, 1, 0)
    df["six"] = np.where(df["batsman_runs"] == 6, 1, 0)
    stats = df.groupby("batsman")[["four", "six"]].sum().sort_values("six", ascending=False).head(top_n)

    fig, ax = plt.subplots()
    stats["six"].sort_values().plot(kind="barh", ax=ax, color="red")
    ax.set_title("Top Batsmen by Sixes")
    ax.set_xlabel("Number of Sixes")
    save_plot(fig, "top_sixes.png")
    plt.close(fig)

    fig, ax = plt.subplots()
    stats["four"].sort_values().plot(kind="barh", ax=ax, color="orange")
    ax.set_title("Top Batsmen by Fours")
    ax.set_xlabel("Number of Fours")
    save_plot(fig, "top_fours.png")
    plt.close(fig)

    return stats

# -------------------- 9. TOSS DECISION --------------------
def toss_decision(matches):
    decision = matches["toss_decision"].value_counts()
    fig, ax = plt.subplots()
    decision.plot.pie(autopct="%1.1f%%", startangle=90, colors=["lightblue", "pink"], ax=ax)
    ax.set_ylabel("")
    ax.set_title("Toss Decision (Bat vs Field)")
    save_plot(fig, "toss_decision.png")
    plt.close(fig)
    return decision

# -------------------- 10. TEAM-WISE TOSS DECISION --------------------
def toss_decision_teamwise(matches):
    teamwise = matches.groupby("toss_winner")["toss_decision"].value_counts().unstack().fillna(0)
    fig, ax = plt.subplots()
    teamwise.plot(kind="bar", stacked=True, ax=ax, colormap="Paired")
    ax.set_title("Team-wise Toss Decisions")
    ax.set_xlabel("Team")
    ax.set_ylabel("Count")
    save_plot(fig, "teamwise_toss_decision.png")
    plt.close(fig)
    return teamwise

# -------------------- 11. MATCHES HOSTED BY CITY --------------------
def matches_by_city(matches):
    city = matches["city"].value_counts().head(15)
    fig, ax = plt.subplots()
    city.sort_values().plot(kind="barh", ax=ax, color="teal")
    ax.set_title("Top 15 Cities Hosting IPL Matches")
    ax.set_xlabel("Matches")
    ax.set_ylabel("City")
    save_plot(fig, "matches_by_city.png")
    plt.close(fig)
    return city

# -------------------- 12. LUCKY STADIUM FOR TEAM --------------------
def lucky_stadium(matches, team):
    team_data = matches[matches["winner"] == team]
    venue = team_data["venue"].value_counts().head(10)
    fig, ax = plt.subplots()
    venue.sort_values().plot(kind="barh", ax=ax, color="violet")
    ax.set_title(f"Top Venues for {team}")
    ax.set_xlabel("Wins")
    ax.set_ylabel("Venue")
    save_plot(fig, f"lucky_stadium_{team}.png")
    plt.close(fig)
    return venue

# -------------------- 13. SEASON-WISE TREND --------------------
def season_trend(matches):
    trend = matches.groupby(["season", "winner"]).size().unstack(fill_value=0)
    fig, ax = plt.subplots()
    trend.plot(ax=ax)
    ax.set_title("Season-wise Wins by Team")
    ax.set_xlabel("Season")
    ax.set_ylabel("Number of Wins")
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
    save_plot(fig, "season_trends.png")
    plt.close(fig)
    return trend

# -------------------- MAIN --------------------
def run_all():
    matches, deliveries, df = load_and_clean()
    print("\nRunning IPL Tasks...\n")

    print("1️⃣ Top Batsmen:")
    print(top_batsmen(df).head(10), "\n")

    print("2️⃣ Top Bowlers:")
    print(top_bowlers(df).head(10), "\n")

    print("3️⃣ Zero Wicket Bowlers:")
    print(zero_wicket_bowlers(df).head(10), "\n")

    print("4️⃣ Top Fielders:")
    print(top_fielders(df).head(10), "\n")

    print("5️⃣ Toss vs Match Winner:")
    print(toss_vs_match(matches), "\n")

    print("6️⃣ Most Successful Teams:")
    print(team_wins(matches).head(10), "\n")

    print("7️⃣ Sixes and Fours:")
    print(fours_sixes(df).head(10), "\n")

    print("8️⃣ Toss Decision:")
    print(toss_decision(matches), "\n")

    print("9️⃣ Team-wise Toss Decision:")
    print(toss_decision_teamwise(matches).head(10), "\n")

    print("🔟 Matches by City:")
    print(matches_by_city(matches), "\n")

    top_team = matches["winner"].value_counts().idxmax()
    print(f"1️⃣1️⃣ Lucky Stadium for {top_team}:")
    print(lucky_stadium(matches, top_team), "\n")

    print("1️⃣2️⃣ Season Trend:")
    print(season_trend(matches).head(), "\n")

    print("✅ All tasks completed! Plots saved inside 'plots' folder.")

# -------------------- RUN --------------------
if __name__ == "__main__":
    run_all()


Matches: (636, 18), Deliveries: (150460, 21), Combined: (150460, 39)

Running IPL Tasks...

1️⃣ Top Batsmen:
✅ Plot saved: plots\top_batsmen.png
batsman
SK Raina          4548
V Kohli           4423
RG Sharma         4207
G Gambhir         4132
DA Warner         4014
RV Uthappa        3778
CH Gayle          3651
S Dhawan          3561
MS Dhoni          3560
AB de Villiers    3486
Name: batsman_runs, dtype: int64 

2️⃣ Top Bowlers:
✅ Plot saved: plots\top_bowlers.png
bowler
Harbhajan Singh    2989
A Mishra           2703
SL Malinga         2694
P Kumar            2637
PP Chawla          2594
R Ashwin           2359
Z Khan             2276
R Vinay Kumar      2161
DW Steyn           2159
IK Pathan          2113
Name: player_dismissed, dtype: int64 

3️⃣ Zero Wicket Bowlers:
Series([], Name: Zero Wicket Bowlers, dtype: object) 

4️⃣ Top Fielders:
✅ Plot saved: plots\top_fielders.png
fielder
KD Karthik        127
MS Dhoni          126
RV Uthappa        115
AB de Villiers    101
SK Raina    